In [13]:

from jqdata import *
from jqlib.technical_analysis import *
from jqfactor import get_factor_values
from jqfactor import winsorize_med
from jqfactor import standardlize
from jqfactor import neutralize
import datetime
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels import regression
from six import StringIO
#导入pca
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from tqdm import tqdm
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
#获取指定周期的日期列表 'W、M、Q'
def get_period_date(peroid,start_date, end_date):
    #设定转换周期period_type  转换为周是'W',月'M',季度线'Q',五分钟'5min',12天'12D'
    stock_data = get_price('000001.XSHE',start_date,end_date,'daily',fields=['close'])
    stock_data['date']=stock_data.index
    period_stock_data=stock_data.resample(peroid,how='last')#how='last'，参数表示在每个周期的最后时刻取值，这意味着如果是周，则取每周的最后一天的收盘价。
    period_stock_data=period_stock_data.set_index('date').dropna()
    date=period_stock_data.index
    pydate_array=date.to_pydatetime()
    date_only_array=np.vectorize(lambda s: s.strftime('%Y-%m-%d'))(pydate_array)
    date_only_series=pd.Series(date_only_array)
    start_date=datetime.datetime.strptime(start_date, "%Y-%m-%d")
    start_date=start_date-datetime.timedelta(days=1)
    start_date=start_date.strftime("%Y-%m-%d")
    date_list=date_only_series.values.tolist()
    date_list.insert(0,start_date)
    return date_list
peroid = 'W'
start_date = '2009-01-01'
# start_date = '2022-01-01'
end_date = '2024-01-01'
DAY = get_period_date(peroid,start_date,end_date)
print(len(DAY),DAY)#周期转换，每周取当周的最后一天作为实际需要交易的日期
#去除上市距beginDate不足3个月的股票
def delect_stop(stocks,beginDate,n=30*3):
    stockList=[]
    beginDate = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
    for stock in stocks:
        start_date=get_security_info(stock).start_date
        if start_date<(beginDate-datetime.timedelta(days=n)).date():
            stockList.append(stock)
    return stockList
#获取股票池
def get_stock(stockPool,begin_date):
    if stockPool=='HS300':
        stockList=get_index_stocks('000300.XSHG',begin_date)
    elif stockPool=='ZZ500':
        stockList=get_index_stocks('399905.XSHE',begin_date)
    elif stockPool=='ZZ800':
        stockList=get_index_stocks('399906.XSHE',begin_date)   
    elif stockPool=='CYBZ':
        stockList=get_index_stocks('399006.XSHE',begin_date)
    elif stockPool=='ZXBZ':
        stockList=get_index_stocks('399005.XSHE',begin_date)
    elif stockPool=='A':
        stockList=get_index_stocks('000002.XSHG',begin_date)+get_index_stocks('399107.XSHE',begin_date)
        stockList = [stock for stock in stockList if not stock.startswith(('68', '4', '8'))]
    elif stockPool=='AA':
        stockList=get_index_stocks('000985.XSHG',begin_date)
        stockList = [stock for stock in stockList if not stock.startswith(('3', '68', '4', '8'))]
    elif stockPool=='small':
        stockList=get_index_stocks('399101.XSHE',begin_date)
        stockList = [stock for stock in stockList if not stock.startswith(('68', '4', '8'))]
    elif stockPool=='small_25':
        initial_list=get_index_stocks('000002.XSHG',begin_date)+get_index_stocks('399107.XSHE',begin_date)
        stockList = list(get_fundamentals(
        query(valuation.code,valuation.market_cap).filter(
            valuation.code.in_(initial_list),
#             valuation.market_cap < 25
        ).order_by(
            valuation.circulating_market_cap.asc()
        )).code)[:50]#之前收益高是因为这里限制的严格
        
    
    #剔除ST股
    st_data=get_extras('is_st',stockList, count = 1,end_date=begin_date)
    stockList = [stock for stock in stockList if not st_data[stock][0]]
    #剔除停牌、新股及退市股票
    stockList=delect_stop(stockList,begin_date)
    return stockList
#获取时间为date的全部因子数据
def get_factor_data(securities_list,date,jqfactors_list):
    factor_data = get_factor_values(securities=securities_list, \
                                    factors=jqfactors_list, \
                                    count=1, \
                                    end_date=date)
    df_jq_factor=pd.DataFrame(index=securities_list)
    for i in factor_data.keys():
        df_jq_factor[i]=factor_data[i].iloc[0,:]
    return df_jq_factor
dateList = get_period_date(peroid,start_date, end_date)
print("交易日长度",len(dateList))

767 ['2008-12-31', '2009-01-09', '2009-01-16', '2009-01-23', '2009-02-06', '2009-02-13', '2009-02-20', '2009-02-27', '2009-03-06', '2009-03-13', '2009-03-20', '2009-03-27', '2009-04-03', '2009-04-10', '2009-04-17', '2009-04-24', '2009-04-30', '2009-05-08', '2009-05-15', '2009-05-22', '2009-05-27', '2009-06-05', '2009-06-12', '2009-06-19', '2009-06-26', '2009-07-03', '2009-07-10', '2009-07-17', '2009-07-24', '2009-07-31', '2009-08-07', '2009-08-14', '2009-08-21', '2009-08-28', '2009-09-04', '2009-09-11', '2009-09-18', '2009-09-25', '2009-09-30', '2009-10-09', '2009-10-16', '2009-10-23', '2009-10-30', '2009-11-06', '2009-11-13', '2009-11-20', '2009-11-27', '2009-12-04', '2009-12-11', '2009-12-18', '2009-12-25', '2009-12-31', '2010-01-08', '2010-01-15', '2010-01-22', '2010-01-29', '2010-02-05', '2010-02-12', '2010-02-26', '2010-03-05', '2010-03-12', '2010-03-19', '2010-03-26', '2010-04-02', '2010-04-09', '2010-04-16', '2010-04-23', '2010-04-30', '2010-05-07', '2010-05-14', '2010-05-21', '

In [ ]:
#训练数据
jqfactors_lists=[
    [
#     "EMA5",
#     "EMAC10",
#     "EMAC12",
#     "EMAC20",
#     "EMAC26",
    "EMAC120",#大的好一般比较有价值
#     'AR',#人气指标
#     "BR",#意愿指标
    "BBIC",#BBI动量
    "BIAS5",#5日乖离率
    "CCI10",#10日顺势指标
    ],
#     [
#     'AR',#人气指标
#     "BR",#意愿指标
#     'ARBR',#情绪类因子 ARBR
#     'VOL120',#情绪类因子 120日平均换手率
#     'DAVOL20',#情绪类因子 20日平均换手率与120日平均换手率之比
#     'TVSTD6',#情绪类因子 6日成交金额的标准差
#     'money_flow_20',#20日资金流量
#     'sharpe_ratio_120',#风险类因子 120日夏普率
#     ],
#     [
#     'ARBR', #情绪类因子 ARBR
#     'SGAI', #质量类因子 销售管理费用指数
#     'net_profit_to_total_operate_revenue_ttm', #质量类因子 净利润与营业总收入之比
#     'retained_profit_per_share' #每股指标因子 每股未分配利润
#     ],
#     [
#     'Price1Y', #动量类因子 当前股价除以过去一年股价均值再减1
#     'total_profit_to_cost_ratio', #质量类因子 成本费用利润率
#     'VOL120' #情绪类因子 120日平均换手率
#     ],
    [
    'price_no_fq', #技术指标因子 不复权价格因子
    'total_profit_to_cost_ratio', #质量类因子 成本费用利润率
    'inventory_turnover_rate' #质量类因子 存货周转率
    ],
    [
    'debt_to_assets', #风格因子 资产负债率
    'operating_cost_to_operating_revenue_ratio', #质量类因子 销售成本率
    'DAVOL20', #情绪类因子 20日平均换手率与120日平均换手率之比
    'price_no_fq', #技术指标因子 不复权价格因子
    'sales_growth' #风格因子 5年营业收入增长率
    ],
    [
    'TVSTD6', #情绪类因子 6日成交金额的标准差
    'cashflow_per_share_ttm', #每股指标因子 每股现金流量净额
    'sharpe_ratio_120', #风险类因子 120日夏普率
    'non_operating_net_profit_ttm' #基础科目及衍生类因子 营业外收支净额TTM
    ]
]
jqfactors_list=[]
for jqfactors in jqfactors_lists:
    jqfactors_list+=jqfactors
print(jqfactors_list)


#定义数据处理方式
def make_train_data(train_data,date):
        stockList=get_stock('small_25',date)
        print(len(stockList))
        factor_solve_data = get_factor_data(stockList,date,jqfactors_list)
        factor_solve_data=factor_solve_data.reset_index(drop=False)
#         factor_solve_data[factor_solve_data.columns[0]] = factor_solve_data[factor_solve_data.columns[0]].rename('code')
        factor_solve_data["code"]=factor_solve_data["index"]
#         print("factor_solve_data",factor_solve_data)
#         factor_solve_data.to_csv("factor_solve_data.csv")
        data_close=get_price(stockList,date,dateList[dateList.index(date)+1],
                             frequency='1d',
                             fields=['close','high_limit'],
                             panel=False,
                             fill_paused=False)
#         print("data_close",data_close)
        data_close=data_close.sort_values(by='time',ascending=True)#ascending=True从小到大排序
        data_close=data_close.groupby("code",group_keys=False).apply(lambda x: x.assign(
                **{f"pchg": (x["close"].iloc[-1]/x["close"].iloc[0]-1)}
                ))
        data_close=data_close.groupby("code").apply(lambda x: x.loc[x["time"].idxmax()]).reset_index(drop=True)
#         print("data_close",data_close)
        factor_solve_data=factor_solve_data.merge(data_close[["pchg","code"]],on="code")
#         print("factor_solve_data",factor_solve_data)
        factor_solve_data['date']=date#日期
#         print(factor_solve_data)
        if train_data.empty:
            train_data=factor_solve_data
        else:
            train_data=train_data.append(factor_solve_data)
#         print(train_data)
        return train_data
#训练数据
train_data=pd.DataFrame()
train_length = math.floor(len(dateList)*0.800)#0.8的训练数据0.2的测试数据
# train_length = math.floor(len(dateList)*0.999)#
for date in tqdm(dateList[:train_length]):
    try:
        train_data=make_train_data(train_data,date)
    except Exception as e:
        print(f"发生bug: {e}")
train_data.to_csv(r'train_baseline_2.0_1.csv')
print(len(train_data),train_data)
# 测试数据
train_data=pd.DataFrame()
for date in tqdm(dateList[train_length:-1]):
    try:
        train_data=make_train_data(train_data,date)
    except Exception as e:
        print(f"发生bug: {e}")
train_data.to_csv(r'test_baseline_2.0_1.csv')
print(len(train_data))

['EMAC120', 'BBIC', 'BIAS5', 'CCI10', 'price_no_fq', 'total_profit_to_cost_ratio', 'inventory_turnover_rate', 'debt_to_assets', 'operating_cost_to_operating_revenue_ratio', 'DAVOL20', 'price_no_fq', 'sales_growth', 'TVSTD6', 'cashflow_per_share_ttm', 'sharpe_ratio_120', 'non_operating_net_profit_ttm']


  0%|          | 0/613 [00:00<?, ?it/s]

40


  0%|          | 1/613 [00:01<11:17,  1.11s/it]

40


...

47


 61%|██████    | 373/613 [05:31<03:42,  1.08it/s]

47


 61%|██████    | 374/613 [05:31<03:38,  1.09it/s]

47


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pickle
from xgboost import XGBClassifier ,XGBRegressor
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pickle
from xgboost import XGBClassifier,XGBRegressor
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score,confusion_matrix,recall_score,precision_score
from sklearn.metrics import precision_recall_curve,auc
import lightgbm as lgb 
import xgboost as xgb 
import joblib  
from sklearn.linear_model import Ridge,BayesianRidge
from sklearn.metrics import mean_squared_error,r2_score

# X_train = pd.read_csv('train_baseline_2.0_1.csv').fillna(0)
# X_test = pd.read_csv('test_baseline_2.0_1.csv').fillna(0)
X_train = pd.read_csv('train_baseline_2.0_1.csv')
X_test = pd.read_csv('test_baseline_2.0_1.csv')
columns_to_fill = [col for col in jqfactors_list if col in X_train.columns]
df_filled_train = X_train.copy()
for column in columns_to_fill:
    df_filled_train[column] = X_train.groupby('date')[column].transform(lambda x: x.fillna(x.median()))
X_train = df_filled_train.dropna()
print(X_train)
df_filled_test = X_test.copy()
for column in columns_to_fill:
    df_filled_test[column] = X_test.groupby('date')[column].transform(lambda x: x.fillna(x.median()))
X_test = df_filled_test.dropna()

y_train = X_train['pchg']
y_test =X_test['pchg']
print(y_train,y_test)

predictions=[]
for n in range(len(jqfactors_lists)+1):
    if n <len(jqfactors_lists):
        this_X_train = X_train[jqfactors_lists[n]]
        this_X_test = X_test[jqfactors_lists[n]]
        Bridge = BayesianRidge()
        Bridge.fit(this_X_train,y_train)
        print(f"jqfactors_lists[n]",jqfactors_lists[n],Bridge.coef_)
        bridge_manual_predictions = np.dot(this_X_test, Bridge.coef_)
        predictions.append(bridge_manual_predictions)
    elif n==len(jqfactors_lists):
        this_X_train = X_train[jqfactors_list]
        this_X_test = X_test[jqfactors_list]
        Bridge = BayesianRidge()
        Bridge.fit(this_X_train,y_train)
        print(f"jqfactors_list",jqfactors_list,Bridge.coef_)
        bridge_manual_predictions = np.dot(this_X_test, Bridge.coef_)
        predictions.append(bridge_manual_predictions)
    bridge_manual_mse = mean_squared_error(y_test, bridge_manual_predictions)#测试值和预测值的比较
    bridge_manual_r2 = r2_score(y_test, bridge_manual_predictions)
    print("\nBayesian Ridge Regression (Manual Prediction):")
    print("Mean Squared Error:", bridge_manual_mse)
    print("R^2 Score:", bridge_manual_r2)#这个是评估因子拟合效果的方法，就是拿因子加权处理之后，判断误差的大小
    #如果出现了方差比较大的问题，就是因子对行情的拟合效果比较差
    
def calculate_layered_returns(result_df, returns_df, num_layers=20):
    backtest_results = pd.DataFrame(columns=[f'Group_{i}' for i in range(1, num_layers + 1)])
    for date in result_df.index:
        # 获取当天的因子值和收益率
        factors = result_df.loc[date]
        returns = returns_df.loc[date]
        # 合并因子值和收益率到一个DataFrame中
        combined_data = pd.DataFrame({'Factor': factors, 'Returns': returns})
        # 按因子值排序
        combined_data = combined_data.sort_values(by='Factor', ascending=False)
        # 分成num_layers组，每组数量相等
        group_size = len(combined_data) // num_layers
        groups = []
        for i in range(num_layers):
            start_idx = i * group_size
            end_idx = (i + 1) * group_size
            group = combined_data.iloc[start_idx:end_idx]
            groups.append(group)
        # 计算每个组的累积收益率
        group_returns = []
        for group in groups:
            group_return = group['Returns'].sum()
            group_returns.append(group_return)
        # 将每个组的累积收益率添加到结果DataFrame中
        backtest_results.loc[date] = [i/group_size for i in group_returns]
    return backtest_results

X_test.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
for i in range(0,len(predictions)):
    y_pred_proba =predictions[i]#最后那一组全包含的，也在这里面序号最后一个，不用单独区分，只要前面append了就行
    X_test['factor'] = y_pred_proba
    factor = X_test.set_index(['id', 'date'])['factor']
    factor = factor.unstack(level='id')
    ret = X_test.set_index(['id', 'date'])['pchg']
    ret = ret.unstack(level='id')
    backtest_results = calculate_layered_returns(factor, ret, num_layers=10)#设置的num_layers分组数量为10
    backtest_results.cumsum().plot()#按说打出来的图里面得分最高到最低跟收益最高到最低的分布比较一致的好
